In [3]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementClickInterceptedException

5성급 호텔 리뷰 웹크롤링

In [499]:
# 크롬 접속
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [4]:
# 아고다 접속
url = 'https://www.agoda.com/ko-kr/'
driver.get(url)

In [5]:
# 광고 제거
driver.find_element('xpath','/html/body/div[12]/div[2]/div[3]/button[2]').click()

In [6]:
# 검색창 클릭
driver.find_element('xpath','//*[@id="textInput"]').click()

In [7]:
# 서울 입력
driver.find_element('xpath','//*[@id="textInput"]').send_keys('서울')

In [8]:
# 서울 클릭
driver.find_element('xpath','//*[@id="SearchBoxContainer"]/div[1]/div/div[2]/div/div/div[6]/div/div/ul/li[1]').click()

In [9]:
# 날짜 자동입력

actions = ActionChains(driver)

actions.send_keys(Keys.SPACE).perform()
actions.send_keys(Keys.SPACE).perform()

In [10]:
# 검색하기
driver.find_element('xpath','//*[@id="SearchBoxContainer"]/div[2]/div/button').click()

In [11]:
# 탭 이동
driver.switch_to.window(driver.window_handles[0])

# 탭 닫기
driver.close()

# 탭 바꾸기
driver.switch_to.window(driver.window_handles[0])

In [12]:
# 호텔 체크
driver.find_element(By.CSS_SELECTOR,'#SideBarLocationFilters > div:nth-child(2) > div.filter-items > ul > li:nth-child(1) > span > span:nth-child(1) > span').click()

In [14]:
# 5성급 체크
driver.find_element(By.CSS_SELECTOR,'#SideBarLocationFilters > div:nth-child(3) > div.filter-items > ul > li:nth-child(1) > span > span:nth-child(1) > span').click()

In [15]:
# 스크롤 다운
actions = ActionChains(driver)
last_height = driver.execute_script('return document.body.scrollHeight')
while True:
    for _ in range (15):
        actions.send_keys(Keys.SPACE).perform()
        time.sleep(1)
    new_height = driver.execute_script('return document.body.scrollHeight')
    if new_height == last_height:
        break
    last_height = new_height


In [16]:
# 호텔명 가져오기
names = []

name_elements = driver.find_elements(By.CSS_SELECTOR, '.sc-jrAGrp.sc-kEjbxe.eDlaBj.dscgss')

for element in name_elements:
    text = element.text
    names.append(text)

len(names)

37

In [17]:
#호텔 url 가져오기
urls = []

url_elements = driver.find_elements(By.CLASS_NAME, 'PropertyCard__Link')
for element in url_elements:
    url = element.get_attribute('href')
    urls.append(url)
len(urls)

37

In [18]:
df = pd.DataFrame({
    'name':names,
    'url':urls,
    'address':None,
    'rating':None,
    'review_rating':None,
    'review_star' : None,
    'review':None
})

In [550]:

# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

In [134]:
# 주소, 별점, 리뷰 가져오기

address = []
scores = []
reviews = []

for i in range(10,11):
    # 호텔 url 들어가기 
    driver.get(urls[i])
    time.sleep(6)
    
    # 주소 가져오기
    try:
        address.append(driver.find_element('xpath','//*[@id="property-main-content"]/div[1]/div[2]/div[2]/span[1]').text)
    except ElementClickInterceptedException:
        continue
    
    # 이용후기 누르기
    try:
        driver.find_element(By.XPATH, '//*[@id="hotelNavBar"]/div/ul/li[4]/button/span').click()
        time.sleep(2)
    except ElementClickInterceptedException:
        continue
    
    # 언어 변경
    try:
        language_dropdown = Select(driver.find_element(By.CSS_SELECTOR, '#reviewFilterSection > div.sc-bdfBwQ.sc-gsTCUz.gdcQLK > div.sc-bdfBwQ.sc-gsTCUz.djZOQg > div > div > label > div.sc-bdfBwQ.sc-gsTCUz.bqqCNI > span > select'))
        # driver.find_element(By.XPATH, '//*[@id="reviewFilterSection"]/div[1]/div[3]/span/i[2]').click() #21번 전까지
        time.sleep(2)
    except ElementClickInterceptedException:
        continue
    
    # "한국어"를 선택
    try:
        language_dropdown.select_by_visible_text("한국어")
        # driver.find_element(By.XPATH, '//*[@id="reviews-language-filter_list"]/ul/li[2]').click() # 21번 전까지
        time.sleep(2)
    except ElementClickInterceptedException:
        continue
    
    while True:
        # 별점 가져오기
        star_elements = driver.find_elements(By.CLASS_NAME,'Review-comment-leftScore')
        for element in star_elements:
            score = element.text
            scores.append(score)
            
        # 리뷰 가져오기
        review_elements = driver.find_elements(By.CLASS_NAME,'Review-comment-bodyText')
        for element in review_elements:
            review = element.text
            reviews.append(review)
            
        # 페이지 넘기기
        try:
            driver.find_element('xpath','//*[@id="reviewSection"]/div[4]/div/span[3]/i').click()
            time.sleep(2)
        except ElementClickInterceptedException:
            # 다음 페이지가 없으면 while 루프 종료
            break
        except NoSuchElementException:
            break
    
    # 데이터 추가하기
    if len(address) == 1:
        df.iloc[i]['address'] = address
        address = []
     
    if len(scores) > 0:
        df.iloc[i]['review_rating'] = int(sum(map(float,scores))) / len(scores)
        df.iloc[i]['review_star'] = scores
        scores = []
    
    if len(reviews) > 0:
        df.iloc[i]['review'] = reviews
        reviews = []
    
    # 다음 호텔로 넘어가기
    continue

In [169]:
# 데이터 따로 입력해주기

stars = []
address = []
scores = []
reviews = []

for i in range(14,15):
    # 호텔 url 들어가기 
    driver.get(urls[i])
    time.sleep(10)
    
    # 주소
    address.append(driver.find_element('xpath','//*[@id="property-main-content"]/div[1]/div[2]/div[2]/span[1]').text)
        
    while True:
        # 별점 가져오기
        star_elements = driver.find_elements(By.CLASS_NAME,'Review-comment-leftScore')
        for element in star_elements:
            score = element.text
            scores.append(score)
            
        # 리뷰 가져오기
        review_elements = driver.find_elements(By.CLASS_NAME,'Review-comment-bodyText')
        for element in review_elements:
            review = element.text
            reviews.append(review)
            
        # 페이지 넘기기
        try:
            driver.find_element('xpath','//*[@id="reviewSection"]/div[3]/div/span[3]/i').click()
            time.sleep(2)
        except ElementClickInterceptedException:
            # 다음 페이지가 없으면 while 루프 종료
            break

    # 데이터 추가하기
    if len(address) == 1:
        df.iloc[i]['address'] = address
        address = []
     
    if len(scores) > 0:
        df.iloc[i]['review_rating'] = int(sum(map(float,scores))) / len(scores)
        df.iloc[i]['review_star'] = scores
        scores = []
    
    if len(reviews) > 0:
        df.iloc[i]['review'] = reviews
        reviews = []
        
    # 다음 호텔로 넘어가기
    continue

In [92]:
# 데이터 추가하기
if len(address) == 1:
    df.iloc[i]['address'] = address
    address = []
    
if len(scores) > 0:
    df.iloc[i]['review_rating'] = int(sum(map(float,scores))) / len(scores)
    df.iloc[i]['review_star'] = scores
    scores = []

if len(reviews) > 0:
    df.iloc[i]['review'] = reviews
    reviews = []

In [174]:
stars = []

for i in range(len(urls)):
    # 호텔 url 들어가기 
    driver.get(urls[i])
    time.sleep(2)
    
    # 홈페이지 총점 가져오기
    try :
        stars.append(float(driver.find_element('xpath','//*[@id="property-critical-root"]/div/div[5]/div[2]/div[1]/div[1]/div/div[1]/div/div/div/div/div/div/p[1]').text))
    except NoSuchElementException:
        stars.append('')

# 데이터 추가하기
df['rating'] = stars

In [178]:
df.to_csv('./data/Five_Stars_Hotels.csv',index=False)

In [167]:
answer = []
for i in range(len(urls)):
    if df.iloc[i]['review'] is None or df.iloc[i]['review_star'] is None:
        continue
    elif len(df.iloc[i]['review']) != len(df.iloc[i]['review_star']):
        answer.append(i)
answer

[10, 12]

In [165]:
len(df.iloc[10]['review'])

294

In [166]:
len(df.iloc[10]['review_star'])

300

===================================================================================================================================

4성급 호텔 웹크롤링

같은 방식으로 접근

In [13]:
# 4성급 누르기
driver.find_element('xpath','//*[@id="SideBarLocationFilters"]/div[3]/div[2]/ul/li[2]/span/span[1]/span').click()

In [14]:
# 스크롤 다운
actions = ActionChains(driver)
last_height = driver.execute_script('return document.body.scrollHeight')
while True:
    for _ in range (15):
        actions.send_keys(Keys.SPACE).perform()
        time.sleep(1)
    new_height = driver.execute_script('return document.body.scrollHeight')
    if new_height == last_height:
        break
    last_height = new_height


In [15]:
# 호텔명 가져오기
names = []

name_elements = driver.find_elements(By.CSS_SELECTOR, '.sc-jrAGrp.sc-kEjbxe.eDlaBj.dscgss')

for element in name_elements:
    text = element.text
    names.append(text)

len(names)

99

In [16]:
#호텔 url 가져오기
urls = []

url_elements = driver.find_elements(By.CLASS_NAME, 'PropertyCard__Link')
for element in url_elements:
    url = element.get_attribute('href')
    urls.append(url)
len(urls)

99

In [17]:
df = pd.DataFrame({
    'name':names,
    'url':urls,
    'address':None,
    'rating':None,
    'review_rating':None,
    'review_star' : None,
    'review':None
})

In [ ]:
# 주소, 별점, 리뷰 가져오기

address = []
scores = []
reviews = []

for i in range(89,len(urls)):
    # 호텔 url 들어가기 
    driver.get(urls[i])
    time.sleep(6)
    
    # 주소 가져오기
    try:
        address.append(driver.find_element('xpath','//*[@id="property-main-content"]/div[1]/div[2]/div[2]/span[1]').text)
    except ElementClickInterceptedException:
        continue
    
    # 이용후기 누르기
    try:
        driver.find_element(By.XPATH, '//*[@id="hotelNavBar"]/div/ul/li[4]/button/span').click()
        time.sleep(2)
    except ElementClickInterceptedException:
        continue
    
    # 언어 변경
    try:
        language_dropdown = Select(driver.find_element(By.CSS_SELECTOR, '#reviewFilterSection > div.sc-bdfBwQ.sc-gsTCUz.gdcQLK > div.sc-bdfBwQ.sc-gsTCUz.djZOQg > div > div > label > div.sc-bdfBwQ.sc-gsTCUz.bqqCNI > span > select'))
        # driver.find_element(By.XPATH, '//*[@id="reviewFilterSection"]/div[1]/div[3]/span/i[2]').click() #21번 전까지
        time.sleep(1)
    except ElementClickInterceptedException:
        continue
    
    # "한국어"를 선택
    try:
        language_dropdown.select_by_visible_text("한국어")
        # driver.find_element(By.XPATH, '//*[@id="reviews-language-filter_list"]/ul/li[2]').click() # 21번 전까지
        time.sleep(1)
    except ElementClickInterceptedException:
        continue
    
    while True:
        # 별점 가져오기
        star_elements = driver.find_elements(By.CLASS_NAME,'Review-comment-leftScore')
        for element in star_elements:
            score = element.text
            scores.append(score)
            
        # 리뷰 가져오기
        review_elements = driver.find_elements(By.CLASS_NAME,'Review-comment-bodyText')
        for element in review_elements:
            review = element.text
            reviews.append(review)
            
        # 페이지 넘기기
        try:
            driver.find_element('xpath','//*[@id="reviewSection"]/div[4]/div/span[3]/i').click()
            time.sleep(1)
        except ElementClickInterceptedException:
            # 다음 페이지가 없으면 while 루프 종료
            break
        
    # 데이터 추가하기
    if len(address) == 1:
        df.iloc[i]['address'] = address
        address = []
     
    if len(scores) > 0:
        df.iloc[i]['review_rating'] = int(sum(map(float,scores))) / len(scores)
        df.iloc[i]['review_star'] = scores
        scores = []
    
    if len(reviews) > 0:
        df.iloc[i]['review'] = reviews
        reviews = []
    
    # 다음 호텔로 넘어가기
    continue

In [515]:
# 데이터 따로 입력해주기

stars = []
address = []
scores = []
reviews = []

for i in range(69,70):
    # 호텔 url 들어가기 
    driver.get(urls[i])
    time.sleep(10)
    
    # 주소
    address.append(driver.find_element('xpath','//*[@id="property-main-content"]/div[1]/div[2]/div[2]/span[1]').text)
        
    while True:
        # 별점 가져오기
        star_elements = driver.find_elements(By.CLASS_NAME,'Review-comment-leftScore')
        for element in star_elements:
            score = element.text
            scores.append(score)
            
        # 리뷰 가져오기
        review_elements = driver.find_elements(By.CLASS_NAME,'Review-comment-bodyText')
        for element in review_elements:
            review = element.text
            reviews.append(review)
            
        # 페이지 넘기기
        try:
            driver.find_element('xpath','//*[@id="reviewSection"]/div[4]/div/span[3]/i').click()
            time.sleep(1)
        except ElementClickInterceptedException:
            # 다음 페이지가 없으면 while 루프 종료
            break

    # 데이터 추가하기
    if len(address) == 1:
        df.at[i,'address'] = address
        address = []
     
    if len(scores) > 0:
        df.at[i,'review_rating'] = int(sum(map(float,scores))) / len(scores)
        df.at[i,'review_star'] = scores
        scores = []
    
    if len(reviews) > 0:
        df.at[i,'review'] = reviews
        reviews = []
    
    # 다음 호텔로 넘어가기
    continue

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="reviewSection"]/div[4]/div/span[3]/i"}
  (Session info: chrome=120.0.6099.110); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00536EE3+174339]
	(No symbol) [0x00460A51]
	(No symbol) [0x00176FF6]
	(No symbol) [0x001A9876]
	(No symbol) [0x001A9C2C]
	(No symbol) [0x001DBD42]
	(No symbol) [0x001C7054]
	(No symbol) [0x001DA104]
	(No symbol) [0x001C6DA6]
	(No symbol) [0x001A1034]
	(No symbol) [0x001A1F8D]
	GetHandleVerifier [0x005D4B1C+820540]
	sqlite3_dbdata_init [0x006953EE+653550]
	sqlite3_dbdata_init [0x00694E09+652041]
	sqlite3_dbdata_init [0x006897CC+605388]
	sqlite3_dbdata_init [0x00695D9B+656027]
	(No symbol) [0x0046FE6C]
	(No symbol) [0x004683B8]
	(No symbol) [0x004684DD]
	(No symbol) [0x00455818]
	BaseThreadInitThunk [0x773AFCC9+25]
	RtlG

In [516]:
df[50:]

,name,url,address,rating,review_rating,review_star,review
50,힐튼 가든 인 서울 강남 (Hilton Garden Inn Seoul Gangnam),https://www.agoda.com/ko-kr/hilton-garden-inn-...,"[253 Gangnam Dae-ro Seocho-gu, Seoul, 강남, 서울, ...",8.9,9.072727,"[10.0, 9.6, 10.0, 10.0, 10.0, 9.6, 10.0, 9.6, ...",[바로 앞에 역이 있어서 접근성이 뛰어납니다 객실에 바디워시 치약 칫솔 드라이기 냉...
51,코트야드 서울 타임스퀘어 (Courtyard Seoul Times Square),https://www.agoda.com/ko-kr/courtyard-seoul-ti...,"[영중로 15, 영등포, 서울, 대한민국, 150-798]",8.7,9.148148,"[10.0, 10.0, 9.6, 10.0, 10.0, 10.0, 10.0, 10.0...","[호텔과 모든 편의시설이 다 연결되어있어서 아이 데리고 머무르기도 너무 좋고, 직원..."
52,노보텔 앰배서더 서울 용산 (Novotel Ambassador Seoul Yongsan),https://www.agoda.com/ko-kr/novotel-ambassador...,"[Cheongpa Ro 20 Gil 95, 용산, 서울, 대한민국, 04372]",8.5,8.25,"[9.6, 10.0, 9.2, 10.0, 10.0, 8.0, 9.6, 9.6, 3....","[아이파크몰이 인근에있고, 호텔안에 편의점이있으며, 전혀 불편함없이 호캉스가 가능했..."
53,라까사호텔 서울 (La Casa Hotel Seoul),https://www.agoda.com/ko-kr/la-casa-hotel-seou...,"[강남구 도산대로 1길 83, 강남, 서울, 대한민국, 06027]",8.1,7.54955,"[9.2, 8.8, 10.0, 8.8, 8.8, 7.3, 6.0, 8.7, 7.6,...",[신사역 근처에서 약속 생기거나 늦게까지 놀면 예약하기 좋은 곳이에요! 이쪽 주변엔...
54,밀리오레 호텔 서울 명동 (Migliore Hotel Seoul Myeongdong),https://www.agoda.com/ko-kr/loisir-hotel-seoul...,"[중구 퇴계로 115 (충무로 1가 24-1), 명동, 서울, 대한민국, 100-011]",7.8,7.587065,"[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10....",[주변 호텔 대비해서 저렴해서 혹시나 시설 퀄리티가 떨어지진 않을까 걱정했는데 생가...
55,나인트리 프리미어 로카우스 호텔 서울 용산 (Nine Tree Premier ROK...,https://www.agoda.com/ko-kr/h36845586/hotel/se...,"[서울특별시 용산구 한강대로23길 25, 용산, 서울, 대한민국, 04378]",8.9,9.412322,"[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10....",[가격 대비 만족도 ⭐️⭐️⭐️⭐️⭐️ 깨끗한 객실 내부 시설. 코너뷰 맛집. 오픈...
56,호텔 인 나인 강남 (Hotel in 9 Gangnam),https://www.agoda.com/ko-kr/hotel-in-9-gangnam...,"[영동대로 618, 강남, 서울, 대한민국, 06081]",8.5,8.71028,"[10.0, 10.0, 10.0, 10.0, 9.6, 10.0, 10.0, 10.0...",[지하철 출구에서 가깝고 직원분들도 친절하고 21층에 레스토랑도 있어서 편리했습니다...
57,포포인츠 바이 쉐라톤 조선 서울역 (Four Points by Sheraton Jo...,https://www.agoda.com/ko-kr/four-points-by-she...,"[서울특별시 용산구 한강대로 366 포포인츠 바이 쉐라톤 조선 서울역호텔, 명동, ...",8.6,9.085202,"[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10....",[가족이 서울에 갈일이 있어 숙박할곳을찾다 이호텔이 서울역과 가까워서 예약하게되었습...
58,SR 호텔 사당 (SR hotel Sadang),https://www.agoda.com/ko-kr/karashy-boutique-h...,"[서울시 동작구 사당1동 1043-6, 관악, 서울, 대한민국, 156827]",8.3,8.715232,"[10.0, 10.0, 10.0, 10.0, 9.6, 9.6, 10.0, 10.0,...",[지하철역 하고 접근성도 좋고 깨끗하고 괜찮았습니다.에어컨도 요청하면 바로 켜주시고...
59,글래드 여의도 (GLAD YEOUIDO),https://www.agoda.com/ko-kr/glad-hotel-yeouido...,"[영등포구 의사당대로 16 (여의도동 17-7), 여의도, 서울, 대한민국, 150...",8.9,8.882692,"[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 9.6, 10.0...",[요청사항도 잘들어주시고 진짜 깨끗하고 침대 이불도 깨끗하고 락스냄새도 안나고.. ...


In [398]:
stars = []

for i in range(len(urls)):
    # 호텔 url 들어가기 
    driver.get(urls[i])
    time.sleep(2)
    
    # 홈페이지 총점 가져오기
    try :
        stars.append(float(driver.find_element('xpath','//*[@id="property-critical-root"]/div/div[5]/div[2]/div[1]/div[1]/div/div[1]/div/div/div/div/div/div/p[1]').text))
    except NoSuchElementException:
        stars.append('')

# 데이터 추가하기
df['rating'] = stars

In [519]:
df.to_csv('./data/Four_Stars_Hotels.csv',index=False,encoding='utf-8')

In [520]:
dfdfdf = pd.read_csv('./data/hotel_4star_agoda.csv')
dfdfdf

,Unnamed: 0,hotel_name,hotel_star,hotel_rate,hotel_url,hotel_location,review
0,0,인터시티 서울 호텔 (Intercity Seoul Hotel),4성급,7.7,https://www.agoda.com/ko-kr/intercity-seoul-ho...,"마곡중앙 6로 76-3, 강서, 서울, 대한민국, 157-201","[['10.0', '위치, 청결, 직원들의 상냥한 서비스, 시설 등등 모든것이 좋았..."
1,1,코트야드 서울 보타닉파크 (Courtyard Seoul Botanic Park),4성급,9.0,https://www.agoda.com/ko-kr/courtyard-seoul-bo...,"마곡중앙 12로 10, 강서, 서울, 대한민국, 07789","[['10.0', '서울식물원에 갈 겸 친구들과 예약을 했어요. 체크인 할 때부터 ..."
2,2,나인트리 프리미어 호텔 인사동 (Nine Tree Premier Hotel Insa...,4성급,8.9,https://www.agoda.com/ko-kr/nine-tree-premier-...,"종로구 인사동길 49, 명동, 인사동, 서울, 대한민국, 03145","[['10.0', '어디로든 이동하기 좋은 위치이고 주변에 관광지를 모두 걸어서 이..."
3,3,유니언 호텔 (UNION HOTEL),4성급,9.1,https://www.agoda.com/ko-kr/seoul-yeongdeungpo...,"unionhotel.sy@gmail.com, 영등포, 서울, 대한민국, 07205","[['10.0', '나는 호텔에서 멋진 시간을 보냈습니다. 그 지역은 붐비는 혼잡한..."
4,4,포포인츠 바이 쉐라톤 서울 구로 (Four Points by Sheraton Seo...,4성급,8.4,https://www.agoda.com/ko-kr/best-western-premi...,"구로구 구로동 1128-1, 구로, 서울, 대한민국, 152-050","[['10.0', '호캉스하려고 집근처 가까운데 보던중 예약했습니다. 성인2명 조식..."
...,...,...,...,...,...,...,...
104,104,호텔 엘리에나 (HOTEL ELIENA),4성급,8.7,https://www.agoda.com/ko-kr/h33814979/hotel/se...,"서울 강남구 논현로 645, 강남, 서울, 대한민국, 06116","[['8.0', '깔큼하고 좋았습니다'], ['6.4', '강남 어디든 이용하기좋고..."
105,105,G3 호텔 충무로 (G3 Hotel Chungmuro),4성급,8.9,https://www.agoda.com/ko-kr/g3-hotel-chungmuro...,"서울특별시 중구 충무로 7, 명동, 서울, 대한민국, 100-271","[['10.0', '명동, 을지로, 충무로 인근에 여행/출장이라면 선택할 수 있는 ..."
106,106,솔라리아 니시테츠 호텔 서울 명동 (Solaria Nishitetsu Hotel S...,4성급,9.0,https://www.agoda.com/ko-kr/solaria-nishitetsu...,"명동8길 27, 7-22F, 명동, 서울, 대한민국, 100-809","[['10.0', '먼저, 위치가 완벽합니다. 명동에 위치하여 매우 좋은 레스토랑과..."
107,107,G2 호텔 명동 (G2 Hotel Myeongdong),4성급,8.8,https://www.agoda.com/ko-kr/g2-hotel-myeongdon...,"수표로 24, 명동, 서울, 대한민국, 04555","[['10.0', '아내와 함께 딸을 방문할 때 서울에서 일주일간 이 호텔에 머물렀..."


In [521]:
dfdfdf.iloc[0]['review']

"[['10.0', '위치, 청결, 직원들의 상냥한 서비스, 시설 등등 모든것이 좋았습니다. 그러나 레지던스에 맞지 않게 식기류들이 아예 없었고, 조리도구가 하나도 없더군요. 게다가 냉방이 안되어 창문을 열어놔야하는데 밖에서 새벽까지 들려오는 소음에 잠을 제대로 잘 수가 없었습니다. 장기 투숙을 고민하고 있었지만, 이곳에서 장기투숙은 권하지 않습니다.'], ['9.6', '나혼자 한달을 지냈고, 처음에는 조금 방이 적은 것 같았는데, 지내다 보니까 오히려 장기 숙박자에게는 불편하지 않다고 생각되고 후론트 직원들이 친절해서 상당히 만족스러웠습니다. 사실 한달을 연장할까를 생각하며, 고민도 해봤습니다. 그곳에 지리만 익숙하면 좀더 재미있지 않았을까라는 생각도 들었습니다.'], ['10.0', '위치도 좋고 서비스도 좋았구요 객실 컨디션까지 다 좋았습니다. 단 가운이 없기 때문에 잠옷은 필수에요! 근처 지하철도 있고, 술집, 밥집,카페 모두 다 있습니다.'], ['10.0', '위치도 좋고, 룸 컨디션도 괜찮구요, 인근 먹거리들도 가득해요. 출장오신 분들이나 서울 중심부에 숙소금액이 높아서 부담스러우신 분들에게 추천합니다!'], ['10.0', '리셉션에 항상 직원이 대기하고 있음. 다리미 세트 대여 가능. 짐 위탁 가능 발산역에서 가깝고 주변에 식당도 많다. 화장실 환풍기로 다른 방 담배 냄새가 넘어오는데 실내 흡연 적발시 퇴실로 관리가 나쁘지 않았음. 매일 아침에 문 앞에 물2병, 수건 놔주심. 크리스마스있는 주간 임에도 가격은 많이 안올랐던듯 합니다.'], ['10.0', '항상 밝은 미소로 반겨주셔서 감사합니다.'], ['9.6', '한 사람이 머무르기에 충분히 큰 방입니다. 건물이 오래되어 방 내부를 포함하여 부분적으로 퇴화되어 있지만, 그것을 신경 쓰지 않는다면 매우 좋은 공간입니다. 호텔 인근에는 매우 맛있는 뷔페 스타일의 레스토랑과 편의점이 있습니다. 방 안에 전자 레인지와 조리기구 등도 있으므로 다양한 요리를 해보기도 좋을 것입니다.'], ['10.